In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import re
from sklearn.preprocessing import LabelEncoder

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2022//train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2022//test.csv')
sub = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2022//sample_submission.csv')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
print(train.shape)
print(test.shape)

As You can see target variable is skewed on the Right side

In [ ]:
sns.distplot(train['num_sold'])

Perform Exponential Distribution

In [ ]:
sns.distplot(np.log1p(train['num_sold']))

In [ ]:
df = pd.concat([train,test])

In [ ]:
df['day'] = pd.to_datetime(df.date).dt.day
df['day_w'] = pd.to_datetime(df.date).dt.dayofweek
df['month'] = pd.to_datetime(df.date).dt.month
df['year'] = pd.to_datetime(df.date).dt.year

In [ ]:
def trait_season(x):
  if ((x<=4)&(x>=2)):
    return 0
  elif ((x<=7)&(x>=5)):
    return 1
  elif ((x<=10)&(x>=8)):
    return 2
  else:
    return 3
df['season'] = df['month'].apply(trait_season)

In [ ]:
def trait_weekend(x):
  if x>=5:
    return 1
  else :
    return 0
df['weekend'] = df['day_w'].apply(trait_weekend)

In [ ]:
!pip install holidays

In [ ]:
from datetime import date
import holidays

In [ ]:
f_holidays = holidays.FIN()
n_holidays = holidays.NOR()
s_holidays = holidays.SWE()

In [ ]:
holidays=[]
for k in range(len(df)):
  if df['country'].iloc[k] == 'Finland':
    if df['date'].iloc[k] in f_holidays:
      holidays.append(1)
    else:
      holidays.append(0)
  elif df['country'].iloc[k] == 'Norway':
    if df['date'].iloc[k] in n_holidays:
      holidays.append(1)
    else:
      holidays.append(0)
  else:
    if df['date'].iloc[k] in s_holidays:
      holidays.append(1)
    else:
      holidays.append(0)
df['holidays'] = holidays

In [ ]:
df.groupby('country')['store'].value_counts()

Distribution of Products

In [ ]:
sns.countplot(train['product'])

Revenue For Each country(FIN,NOR,SWED) for every Year(2015-2019)

In [ ]:
rev_f = {"2015":3877.5,"2016":3754.2,"2017":3732.5,"2018":4019.2,"2019":4163.3}
rev_n = {"2015":7119.2,"2016":6865.8,"2017":6304.2,"2018":6693.3,"2019":6801.7}
rev_s = {"2015":4813.3,"2016":4540,"2017":4419.2,"2018":4636.7,"2019":4674.2}

In [ ]:
#PIB
pib_f = {"2015":234.4,"2016":240.6,"2017":255,"2018":275.8,"2019":269}
pib_n = {"2015":385.5,"2016":368.8,"2017":398.4,"2018":437,"2019":405.5}
pib_s = {"2015":505.1,"2016":515.7,"2017":541,"2018":555.5,"2019":531.3}

In [ ]:
inf_f = {"2015":-0.16,"2016":0.39,"2017":0.84,"2018":1.17,"2019":1.33}
inf_n = {"2015":2.17,"2016":3.55,"2017":1.88,"2018":2.77,"2019":2.17}
inf_s = {"2015":0.05,"2016":1.74,"2017":1.74,"2018":2.04,"2019":1.75}

In [ ]:
un_f = {"2015":9.38,"2016":8.79,"2017":8.7,"2018":8.11,"2019":7.78}
un_n = {"2015":4.37,"2016":4.7,"2017":4,"2018":3.8,"2019":3.7}
un_s = {"2015":7.4,"2016":6.95,"2017":6.57,"2018":6.33,"2019":6.3}

In [ ]:
pib_f[str(df['year'].iloc[0])]

In [ ]:
df.head()

In [ ]:
pib=[]
rev=[]
inf=[]
unp=[]
for k in range(len(df)):
  if df['country'].iloc[k] == 'Finland':
    pib.append(pib_f[str(df['year'].iloc[k])])
    rev.append(rev_f[str(df['year'].iloc[k])])
    inf.append(inf_f[str(df['year'].iloc[k])])
    unp.append(un_f[str(df['year'].iloc[k])])
  elif df['country'].iloc[k] == 'Norway':
    pib.append(pib_n[str(df['year'].iloc[k])])
    rev.append(rev_n[str(df['year'].iloc[k])])
    inf.append(inf_n[str(df['year'].iloc[k])])
    unp.append(un_n[str(df['year'].iloc[k])])
  else:
    pib.append(pib_s[str(df['year'].iloc[k])])
    rev.append(rev_s[str(df['year'].iloc[k])])
    inf.append(inf_s[str(df['year'].iloc[k])])
    unp.append(un_s[str(df['year'].iloc[k])])

In [ ]:
df.head()

In [ ]:
df['PIB'] = pib
df['REV'] = rev
df['INF'] = inf
df['UNP']= unp

In [ ]:
df.head()

In [ ]:
df.drop(['row_id','date'],axis=1,inplace=True)

In [ ]:
print(train.shape)

In [ ]:
cols=['country','store','product']
le=LabelEncoder()
for col in cols:
  df[col] = le.fit_transform(df[col])

In [ ]:
df.head()

In [ ]:
train_data = df[:26298]
test_data = df[26298::]
test_data.drop('num_sold',1,inplace=True)
print(train_data.shape)

In [ ]:
train_data.head()

In [ ]:
test_data.head()

In [ ]:
colormap = sns.diverging_palette(120, 10, as_cmap = True)
plt.figure(figsize=(25,25))
sns.heatmap(train_data.corr(), cbar = True,  square = True, annot=True, fmt= '.2f',annot_kws={'size': 12},cmap = colormap, linewidths=0.1, linecolor='white')

Modelling


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = train_data.drop('num_sold',1)
Y = train_data['num_sold']

In [ ]:
x_train,x_val,y_train,y_val=train_test_split(X,Y,test_size=0.15)

In [ ]:
def smape_loss(y_true, y_pred):
    """SMAPE Loss"""
    return np.abs(y_true - y_pred) / (y_true + np.abs(y_pred)) * 200

In [ ]:
y_tr_lg = y_train.apply(np.log1p)
y_val_lg = y_val.apply(np.log1p)

Generating LGBM DataSet

In [ ]:
import lightgbm as lgb
train_data_lg = lgb.Dataset(x_train, label=y_tr_lg)
val_data_lg = lgb.Dataset(x_val, label=y_val_lg)

In [ ]:
params0 = {'objective': 'regression', # Manual optimization
           'force_row_wise': True,
           'max_bin': 400, # need more bins than days in a year
           'verbosity': -1,
           'seed': 1,
           'bagging_seed': 3,
           'feature_fraction_seed': 2,
           'learning_rate': 0.02,
           'lambda_l1': 0,
           'lambda_l2': 1e-2,
           'num_leaves': 9,
           'feature_fraction': 0.710344827586207,
           'bagging_fraction': 0.47931034482758617,
           'bagging_freq': 5,
           'min_child_samples': 20}

In [ ]:
lgb_model_lg = lgb.train(params0, train_data_lg, 25000, valid_sets = [train_data_lg, val_data_lg], verbose_eval=2000)

In [ ]:
pred_val_lg = lgb_model_lg.predict(x_val,num_iteration=lgb_model_lg.best_iteration)

In [ ]:
pred_val2 = np.expm1(pred_val_lg)

In [ ]:
smape = np.mean(smape_loss(y_val, pred_val2))
smape

Test Prediction and Submission

In [ ]:
pred_test_lg = lgb_model_lg.predict(test_data,num_iteration=lgb_model_lg.best_iteration)

In [ ]:
predicted_test = np.expm1(pred_test_lg)

In [ ]:
sub['num_sold'] = predicted_test

In [ ]:
sub.to_csv('lgbm.csv',index=False)

In [ ]:
!pip install catboost

In [ ]:
from catboost import CatBoostRegressor

In [ ]:
cat_model = CatBoostRegressor(iterations= 22000,
        depth= 6, 
        learning_rate=0.019,
        l2_leaf_reg= 12.06,
        bootstrap_type= 'Bayesian',
        boosting_type= 'Plain',
        loss_function= 'MAPE',
        eval_metric= 'SMAPE',
        #od_type= 'Iter',       # type of overfitting detector
        #od_wait= 40,
        has_time= True)

In [ ]:
cat_model.fit(x_train, y_tr_lg,
             eval_set=(x_val, y_val_lg),
             use_best_model=True,
             verbose=2000)

In [ ]:
pred_val_cat = cat_model.predict(x_val)

In [ ]:
pred_val_cat = np.expm1(pred_val_cat)
print('SMAPE : ')
np.mean(smape_loss(y_val, pred_val_cat))

In [ ]:
pred_test = cat_model.predict(test_data)
pred_test = np.expm1(pred_test)

In [ ]:
sub['num_sold']=pred_test
sub.to_csv('cat.csv',index=False)

XGBoost

In [ ]:
from xgboost import XGBRegressor

In [ ]:
xgbmodel = XGBRegressor(colsample_bytree=0.3603, gamma=0.0468, 
                             learning_rate=0.07, max_depth=12, 
                             min_child_weight=1.7817, n_estimators=4000,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
xgbmodel.fit(x_train,y_tr_lg)

In [ ]:
pred_xgb = xgbmodel.predict(x_val)
pred_xgb = np.expm1(pred_xgb)

In [ ]:
print('SMAPE : ')
np.mean(smape_loss(y_val, pred_xgb))

In [ ]:
pred_test_xgb = xgbmodel.predict(test_data)
pred_test_xgb = np.expm1(pred_test_xgb)

In [ ]:
sub['num_sold']=pred_test_xgb
sub.to_csv('xgb.csv',index=False)

Stacking(CatBoost,LGBM,XGBoost)

In [ ]:
from sklearn.ensemble import StackingRegressor

In [ ]:
params0 = {'objective': 'regression', # Manual optimization
           'force_row_wise': True,
           'max_bin': 300, # need more bins than days in a year
           'verbosity': -1,
           'seed': 1,
           'bagging_seed': 3,
           'feature_fraction_seed': 2,
           'learning_rate': 0.018,
           'lambda_l1': 0,
           'lambda_l2': 1e-2,
           'num_leaves': 10,
           'feature_fraction': 0.610344827586207,
           'bagging_fraction': 0.48931034482758617,
           'bagging_freq': 6,
           'min_child_samples': 18}

In [ ]:
estimators = [
     ('XGB',XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.07, max_depth=10, 
                             min_child_weight=1.7817, n_estimators=4000,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1) ),
     ('CAT',CatBoostRegressor(iterations= 22000,
        depth= 8, 
        learning_rate=0.02,
        l2_leaf_reg= 12.06,
        bootstrap_type= 'Bayesian',
        loss_function= 'MAPE',
        eval_metric= 'SMAPE',
        has_time= True))]

In [ ]:
reg = StackingRegressor(
     estimators=estimators,
     final_estimator=lgb.LGBMRegressor(**params0),verbose=100)

In [ ]:
reg.fit(x_train, y_tr_lg)

In [ ]:
pred_stack = reg.predict(x_val)
pred_stack = np.expm1(pred_stack)

In [ ]:
np.mean(smape_loss(y_val, pred_stack))

In [ ]:
final_test = reg.predict(test_data)
final_test = np.expm1(final_test)

In [ ]:
sub['num_sold'] = final_test

In [ ]:
sub.to_csv('stack.csv',index=False)

In [ ]:
df_lgbm = pd.read_csv("./lgbm.csv")
df_cat = pd.read_csv("./cat.csv")
df_xgb = pd.read_csv("./xgb.csv")

In [ ]:
sub['num_sold'] = (df_lgbm['num_sold'] + df_cat['num_sold']+ df_xgb['num_sold'])/3


In [ ]:
sub.to_csv('submission.csv',index=False)